## Excercie chargement des bien-fonds dans PostGIS

## Objectif

Dans cet execrcie, nous allons charger les bien-fonds de la commuene d'Yverdon dans une table de la base PostgreSQL. Les bien-fonds provindrons du service REST de l'ArcGIS server du canton de Vaud. Ils seront téléchargés sous la forme de JSON. 

In [1]:
import os

## Récupération des données

L'url du service REST du canton de vaud est : https://ags.vd.ch/arcgis/rest/services/OGC/wmsVD/MapServer. Cette URL propose  les couches publiques de l'administration cantonale. 

En cliquant sur l'une d'entre elle, il est possible d'obtenir le détail de cette dernière, ainsi que, tout en bas de la page, un lien nommée "Query" permettant de construire une requête pour obtenir les données.

L'url de la couche des bien-fonds est : https://ags.vd.ch/arcgis/rest/services/OGC/wmsVD/MapServer/272


A l'aide de l'interface https://ags.vd.ch/arcgis/rest/services/OGC/wmsVD/MapServer/272, créer ci-desous une variable "url_query" qui contient url pour télécharger le json.

Astuces : 
* Pour affiner le résultat, le numéro de commune vaudois (NO_COM_CANT) est égale à 387 A remplir dans le champ "Where"
* Il faut mettre * dans le champ "Out Fields"
* Il faut sélectionner "GeoJSON" dans le champ "Format"
* les système de coordonnée de sortie doit être le 2056 (MN95)

In [2]:
url_query = None 
print(url_query)

# Corrcetion 
layer_no = "272"
out_format = "geojson"
where_clause = "NO_COM_CANT%3D387"
fields = "*"
epsg = "2056"

url_query = "https://ags.vd.ch/arcgis/rest/services/OGC/wmsVD/MapServer/{layer_no}/query?where={where_clause}&text=&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&distance=&units=esriSRUnit_Foot&relationParam=&outFields={fields}&returnGeometry=true&returnTrueCurves=false&maxAllowableOffset=&geometryPrecision=&outSR={epsg}&havingClause=&returnIdsOnly=false&returnCountOnly=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&returnZ=false&returnM=false&gdbVersion=&historicMoment=&returnDistinctValues=false&resultOffset=&resultRecordCount=&returnExtentOnly=false&datumTransformation=&parameterValues=&rangeValues=&quantizationParameters=&featureEncoding=esriDefault&f={out_format}".format(layer_no=layer_no,where_clause=where_clause,fields=fields,out_format=out_format,epsg=epsg)
print(url_query)

None
https://ags.vd.ch/arcgis/rest/services/OGC/wmsVD/MapServer/272/query?where=NO_COM_CANT%3D387&text=&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&distance=&units=esriSRUnit_Foot&relationParam=&outFields=*&returnGeometry=true&returnTrueCurves=false&maxAllowableOffset=&geometryPrecision=&outSR=2056&havingClause=&returnIdsOnly=false&returnCountOnly=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&returnZ=false&returnM=false&gdbVersion=&historicMoment=&returnDistinctValues=false&resultOffset=&resultRecordCount=&returnExtentOnly=false&datumTransformation=&parameterValues=&rangeValues=&quantizationParameters=&featureEncoding=esriDefault&f=geojson


Pour vérifier que l'url a été correctement construite, il suffit de cliquer sur le lien printé pour voir si l'on obtient un JSON.

Est-ce OK ? si oui, on continue en téléchargant le JSON dans une variable nommée "input_data" puis en imprimant le code status de la requête

In [3]:
import requests

input_data = None # A compléter 

# Corrcetion 
input_data = requests.get(url_query)
print(input_data.status_code)


200


## Connection à la base de données

Nous allons maintenant nous connecter à la base de données à l'aide de psycopg2

In [4]:
import psycopg2

In [5]:
# Remplir ici les variable de connection

host = "localhost"
database = "cfgeo"
user = 'postgres'
password = 'postgres'

In [12]:
conn = psycopg2.connect(host=host,database=database,user=user,password=password)

## Création des la table dans la base de données

Maintenant que nos données sont contenues dans la variable "input_data", nous devont créer la table "bf" dans la base de données PostgreSQL. 

La table doit contenir les colonnes suivantes :

|Nom de l'attibut   |Type   |
|---|---|
|no_cantonal   |character varying(254)   |
|numero   |character varying(10)   |
|genre   |character varying(254)   |
|geom   |geometry(Polygon,2056)   |


Le code SQL de création doit être nommé "create_table_query"

In [14]:


create_table_query = "" # A compléter ici

# Corrcetion 
create_table_query = """

DROP TABLE if exists public.bf;

CREATE TABLE public.bf
(
    no_cantonal character varying(254),
    numero character varying(10),
    genre character varying(254),
    geom geometry(Polygon,2056)
);

ALTER TABLE public.bf
    OWNER to postgres;

-- Index: commune_geom

DROP INDEX if exists public.bf_geom;

CREATE INDEX bf_geom
    ON public.bf USING gist
    (geom)
    TABLESPACE pg_default;

"""



cursor = conn.cursor()
cursor.execute(create_table_query)
conn.commit()


La table s'est-elle correctement  créée ? Je vous invite à controler cela à l'aide pgAdmin ou bien QGIS 

## Enregistrement des données 

Passons dans le vif du sujet, nous avons les données, la table est prête. Nous pouvons enregistrer les bien-fonds dans la table. 

Pour ce faire, nous devons analyser le schéma du json, itérer chaque objet (feature) pour créer une requête SQL pour chaque bien-fonds incluant les attributs définis précédemment ainsi que la geometrie.

<img src="images/schema_json.png" alt="drawing" width="800"/>



Astuces :
* il faut charger le json comme un dictionnaire python.
* puis créer une boucle en itérant dans le niveau "features"
* pour chaque itération créer les attributs correspondant aux colonnes de la table dans le base de données PostgreSQL
* créer la géométrie. pour ce faire, regarder les fonctions suivantes:
    * https://postgis.net/docs/ST_GeomFromGeoJSON.html
    * https://postgis.net/docs/ST_SetSRID.html
    
    (il faudra utiliser ces deux fonctions pour indiquer à postGIS que nous lui transmettons une géométrie au format GeoJSON en 2056 (MN95)
* voici un exemple d'insertion de données à l'aide de psycopg2 


```
    cursor = conn.cursor()
    insert_feature_query = "Votre requête insert"
    try:
        cursor.execute(insert_feature_query) 
        conn.commit()
    except Exception as error:
        print(error)
        conn.rollback()
    cursor.close()
```
 


In [16]:
import json 


# Corrcetion 


cursor = conn.cursor()
for feature in input_data.json()["features"] :
    geometry = feature["geometry"]
    attributes = feature["properties"]
    no_cantonal = attributes["NO_COM_CANT"]
    numero = attributes["NUMERO"]
    genre = attributes["GENRE_TXT"]
    
    
    geometry_geojson = json.dumps(geometry)
    
    
    
    insert_feature_query = "INSERT INTO public.bf (no_cantonal, numero, genre,geom)  VALUES ('{no_cantonal}','{numero}','{genre}',ST_SetSRID(ST_GeomFromGeoJSON('{geom}'),2056));".format(no_cantonal=no_cantonal,numero=numero,genre=genre,geom=geometry_geojson)

    try:
        cursor.execute(insert_feature_query) 
        conn.commit()
    except Exception as error:
        print(error)
        conn.rollback()

cursor.close()
    
    
    



A-t-on récupérer tous les bien-fonds de la commune ? Je vous laisse aller voir cette table avec Qgis.

<img src="images/error_1000.png" alt="drawing" width="800"/>

Eh non, la totalité des bien-fonds n'ont pas été introduite dans notre table. Cela vien du fait que le requête limite le nombre d'objet à 1000. Nous avons que 1000 biens-fond dans la table. Nous devons donc trouver une autre stratégie. 

## Récupération des données par ObjectID

Il existe une requête qui n'est pas limitée à 1000 object, celle qui retour uniquement les identifiants uniques des biens-fonds. 

<img src="images/ids.png" alt="drawing" width="400"/>

La stratégie est la suivante :
* Récupérer tous les identifiants uniques sous forme de liste
* Utiliser les valeurs minumum et maximum des identifiants uniques pour créer des lots de 1000
* Itérer à travers ces lots en les incluant dans les paramètres de l'url (where_clause)

<img src="images/ids2.png" alt="drawing" width="600"/>

Première étape : créer les lots d'object_id. 
Pour ce faire :
* Créer une variable "list_objectids" qui récupère tous les objectids
* Créer une variable "min_objectid" qui prend la valeur minimum de la liste (google est ton ami)
* Créer une variable "max_objectid" qui prend la valeur maximum de la liste (google est ton ami)
* Créer une variable "list_lots_objectids" qui a la forme suivante :

```[[min_objectid_lot_1, max_objectid_lot_1],[min_objectid_lot_2, max_objectid_lot_2], ..., [min_objectid_lot_n, max_objectid_lot_n]```


In [30]:
url_query = None # A modifier
list_objectids = None # A modifier
min_objectid = None  # A modifier
max_objectid = None # A modifier
list_lots_objectids = list() # A modifier


# Corrcetion 
layer_no = "272"
out_format = "geojson"
where_clause = "NO_COM_CANT%3D387"

url_query = "https://ags.vd.ch/arcgis/rest/services/OGC/wmsVD/MapServer/{layer_no}/query?where={where_clause}&text=&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&distance=&units=esriSRUnit_Foot&relationParam=&outFields=&returnGeometry=true&returnTrueCurves=false&maxAllowableOffset=&geometryPrecision=&outSR=&havingClause=&returnIdsOnly=true&returnCountOnly=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&returnZ=false&returnM=false&gdbVersion=&historicMoment=&returnDistinctValues=false&resultOffset=&resultRecordCount=&returnExtentOnly=false&datumTransformation=&parameterValues=&rangeValues=&quantizationParameters=&featureEncoding=esriDefault&f={out_format}".format(layer_no=layer_no,where_clause=where_clause,out_format=out_format)

input_data = requests.get(url_query)
list_objectids = input_data.json()["objectIds"]
min_objectid = min(list_objectids)
max_objectid = max(list_objectids)

for lot_min in range(min_objectid,max_objectid,1000):
    list_temp = list()
    list_temp.append(lot_min)
    list_temp.append(lot_min+1000)
    list_lots_objectids.append(list_temp)
   


Seconde étape : Créer les requêtes de façon itérative en ajoutant la contraite des IDs. Puis créer une variabelnommée "list_input_data" pour sauvegarder le retour des requêtes.

In [33]:
list_input_data = list()

# Corrcetion


layer_no = "272"
out_format = "geojson"
fields = "*"
epsg = "2056"


for lot in list_lots_objectids :
    id_min = lot[0]
    id_max = lot[1]
    where_clause = "NO_COM_CANT=387 and OBJECTID >= {} and OBJECTID < {}".format(id_min,id_max)
    
    url_query = "https://ags.vd.ch/arcgis/rest/services/OGC/wmsVD/MapServer/{layer_no}/query?where={where_clause}&text=&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&distance=&units=esriSRUnit_Foot&relationParam=&outFields={fields}&returnGeometry=true&returnTrueCurves=false&maxAllowableOffset=&geometryPrecision=&outSR={epsg}&havingClause=&returnIdsOnly=false&returnCountOnly=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&returnZ=false&returnM=false&gdbVersion=&historicMoment=&returnDistinctValues=false&resultOffset=&resultRecordCount=&returnExtentOnly=false&datumTransformation=&parameterValues=&rangeValues=&quantizationParameters=&featureEncoding=esriDefault&f={out_format}".format(layer_no=layer_no,where_clause=where_clause,fields=fields,out_format=out_format,epsg=epsg)
    input_data = requests.get(url_query)
    list_input_data.append(input_data)
    

Troisième étape : Maintenant il faut vide la table des bien-fond

In [34]:
create_table_query = "" # A compléter ici

# Corrcetion 
create_table_query = "TRUNCATE public.bf;"


cursor = conn.cursor()
cursor.execute(create_table_query)
conn.commit()

Dernière étape : charger les nouvelles données

In [36]:
import json 


# Corrcetion 


cursor = conn.cursor()

for input_data in list_input_data :
    for feature in input_data.json()["features"] :
        geometry = feature["geometry"]
        attributes = feature["properties"]
        no_cantonal = attributes["NO_COM_CANT"]
        numero = attributes["NUMERO"]
        genre = attributes["GENRE_TXT"]
        geometry_geojson = json.dumps(geometry)

        insert_feature_query = "INSERT INTO public.bf (no_cantonal, numero, genre,geom)  VALUES ('{no_cantonal}','{numero}','{genre}',ST_SetSRID(ST_GeomFromGeoJSON('{geom}'),2056));".format(no_cantonal=no_cantonal,numero=numero,genre=genre,geom=geometry_geojson)

        try:
            cursor.execute(insert_feature_query) 
            conn.commit()
        except Exception as error:
            print(error)
            conn.rollback()

cursor.close()

Et voilà, nos bien-fonds sont totalement chargés ! Bravo

<img src="images/bf_all.png" alt="drawing" width="800"/>



Pour teminer, nous fermons la connection à la base de données

In [9]:
conn.close()